In [6]:
import sys
import numpy as np
import matplotlib
import seaborn as sns
import tqdm
import sklearn
import pandas as pd
from time import time

print ( "Python", sys.version )
print ( "Numpy", np.__version__ )
print ( "Matplotlib", matplotlib.__version__ )
print ( "Seaborn", sns.__version__ )
print ( "tqdm", tqdm.__version__ )
print ( "scikit-learn", sklearn.__version__ )
print ( "pandas", pd.__version__ )

Python 3.7.4 (default, Sep  7 2019, 18:27:02) 
[Clang 10.0.1 (clang-1001.0.46.4)]
Numpy 1.16.1
Matplotlib 3.0.3
Seaborn 0.9.0
tqdm 4.31.1
scikit-learn 0.21.3
pandas 0.24.2


# Traditional Import
- Automatic assign data type by pandas which may cost expensive computational time.

In [9]:
start = time()
df = pd.read_csv(r'./data/avazu-ctr-prediction/sample1m.csv')
t = time() - start
print('Execution time = ', t, ' seconds')
df.info()

Execution time =  4.652196168899536  seconds
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 24 columns):
id                  1000000 non-null float64
click               1000000 non-null int64
hour                1000000 non-null int64
C1                  1000000 non-null int64
banner_pos          1000000 non-null int64
site_id             1000000 non-null object
site_domain         1000000 non-null object
site_category       1000000 non-null object
app_id              1000000 non-null object
app_domain          1000000 non-null object
app_category        1000000 non-null object
device_id           1000000 non-null object
device_ip           1000000 non-null object
device_model        1000000 non-null object
device_type         1000000 non-null int64
device_conn_type    1000000 non-null int64
C14                 1000000 non-null int64
C15                 1000000 non-null int64
C16                 1000000 non-null int64
C17            

# Managing Data Structure Type and Optimize it

In [15]:
# Integer boolean mask
Integer_feature_mask = df.dtypes==int
# filter Integer columns using mask and turn it into a list
Integer_cols = df.columns[Integer_feature_mask].tolist()

# Object boolean mask
Object_feature_mask = df.dtypes==object
# filter Object columns using mask and turn it into a list
Object_cols = df.columns[Object_feature_mask].tolist()

dict_all = {i:np.int8 for i in Integer_cols} 
dict_cat = {i:'category' for i in Object_cols}
dict_all.update(dict_cat)
dict_int = {i:np.int8 for i in Integer_cols} 

exclude_list = ['id', 'click', 'site_id', 'site_domain', 'app_id', 
                'device_id', 'device_ip', 'device_model', 'C14']

exclude_list.append('hour')
col_name = df_sam.columns.tolist()
read_col = [ col for col in col_name if col not in exclude_list]
read_col.append('click')

dict_int_2 = {i:np.int8 for i in Integer_cols} 
# del dict_int_2['hour']
''' 
Deleting an entry from dictionary using del 
'''
# If key exist in dictionary then delete it using del.
for key in dict_int.keys():
    if key not in read_col:
        del dict_int_2[key]

# All Column with Casting int64 > int8
- Becareful with each value of integer column. When you cast int64 to int8 if it has some value out of range 2\**8 = [-128,128] , it could change the original value.

In [17]:
start = time()
df_sam_3 = pd.read_csv('./data/avazu-ctr-prediction/sample1m.csv',
                       dtype=dict_int,
#                        parse_dates=["hour"]
                      )
t = time() - start
print('Execution time = ', t, ' seconds')
df_sam_3.info()

Execution time =  3.8226230144500732  seconds
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 24 columns):
id                  1000000 non-null float64
click               1000000 non-null int8
hour                1000000 non-null int8
C1                  1000000 non-null int8
banner_pos          1000000 non-null int8
site_id             1000000 non-null object
site_domain         1000000 non-null object
site_category       1000000 non-null object
app_id              1000000 non-null object
app_domain          1000000 non-null object
app_category        1000000 non-null object
device_id           1000000 non-null object
device_ip           1000000 non-null object
device_model        1000000 non-null object
device_type         1000000 non-null int8
device_conn_type    1000000 non-null int8
C14                 1000000 non-null int8
C15                 1000000 non-null int8
C16                 1000000 non-null int8
C17                 100

# All Column with Casting int64 > int8, object > category

In [11]:
start = time()
df_sam_2 = pd.read_csv('./data/avazu-ctr-prediction/sample1m.csv',
                       dtype=dict_all,
#                        parse_dates=["hour"]
                      )
t = time() - start
print('Execution time = ', t, ' seconds')
df_sam_2.info()

Execution time =  8.584215879440308  seconds
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 24 columns):
id                  1000000 non-null float64
click               1000000 non-null int8
hour                1000000 non-null int8
C1                  1000000 non-null int8
banner_pos          1000000 non-null int8
site_id             1000000 non-null category
site_domain         1000000 non-null category
site_category       1000000 non-null category
app_id              1000000 non-null category
app_domain          1000000 non-null category
app_category        1000000 non-null category
device_id           1000000 non-null category
device_ip           1000000 non-null category
device_model        1000000 non-null category
device_type         1000000 non-null int8
device_conn_type    1000000 non-null int8
C14                 1000000 non-null int8
C15                 1000000 non-null int8
C16                 1000000 non-null int8
C17   

# Selected Column without Casting

In [20]:
#Read specific columns from CSV:
# https://honingds.com/blog/pandas-read_csv/
# YYMMDDHH
# mydateparser = lambda x: pd.datetime.strptime(x, '%Y%m%d%H' )
 
start = time()
df_sam_4 = pd.read_csv('./data/avazu-ctr-prediction/sample1m.csv',
                       usecols = read_col,
#                        dtype = dict_int_2,
#                        parse_dates=['hour'], date_parser = mydateparser,
                      )
t = time() - start
print('Execution time = ', t, ' seconds')
df_sam_4.info()

Execution time =  2.580280065536499  seconds
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 15 columns):
click               1000000 non-null int64
C1                  1000000 non-null int64
banner_pos          1000000 non-null int64
site_category       1000000 non-null object
app_domain          1000000 non-null object
app_category        1000000 non-null object
device_type         1000000 non-null int64
device_conn_type    1000000 non-null int64
C15                 1000000 non-null int64
C16                 1000000 non-null int64
C17                 1000000 non-null int64
C18                 1000000 non-null int64
C19                 1000000 non-null int64
C20                 1000000 non-null int64
C21                 1000000 non-null int64
dtypes: int64(12), object(3)
memory usage: 114.4+ MB


# Selected Column with Casting int64 > int8

In [18]:
#Read specific columns from CSV:
# https://honingds.com/blog/pandas-read_csv/
# YYMMDDHH
# mydateparser = lambda x: pd.datetime.strptime(x, '%Y%m%d%H' )
 
start = time()
df_sam_5 = pd.read_csv('./data/avazu-ctr-prediction/sample1m.csv',
                       usecols = read_col,
                       dtype = dict_int_2,
#                        parse_dates=['hour'], date_parser = mydateparser,
                      )
t = time() - start
print('Execution time = ', t, ' seconds')
df_sam_5.info()

Execution time =  2.1211137771606445  seconds
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 15 columns):
click               1000000 non-null int8
C1                  1000000 non-null int8
banner_pos          1000000 non-null int8
site_category       1000000 non-null object
app_domain          1000000 non-null object
app_category        1000000 non-null object
device_type         1000000 non-null int8
device_conn_type    1000000 non-null int8
C15                 1000000 non-null int8
C16                 1000000 non-null int8
C17                 1000000 non-null int8
C18                 1000000 non-null int8
C19                 1000000 non-null int8
C20                 1000000 non-null int8
C21                 1000000 non-null int8
dtypes: int8(12), object(3)
memory usage: 34.3+ MB


# Selected Column with Casting int64 > int8, object > category

In [23]:
start = time()
df_sam_6 = pd.read_csv('./data/avazu-ctr-prediction/sample1m.csv',
                       usecols = read_col,
                       dtype = dict_all,
#                        parse_dates=['hour'], date_parser = mydateparser,
                      )
t = time() - start
print('Execution time = ', t, ' seconds')
df_sam_6.info()

Execution time =  2.3405277729034424  seconds
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 15 columns):
click               1000000 non-null int8
C1                  1000000 non-null int8
banner_pos          1000000 non-null int8
site_category       1000000 non-null category
app_domain          1000000 non-null category
app_category        1000000 non-null category
device_type         1000000 non-null int8
device_conn_type    1000000 non-null int8
C15                 1000000 non-null int8
C16                 1000000 non-null int8
C17                 1000000 non-null int8
C18                 1000000 non-null int8
C19                 1000000 non-null int8
C20                 1000000 non-null int8
C21                 1000000 non-null int8
dtypes: category(3), int8(12)
memory usage: 15.3 MB


| <u>*Import 1,000,000 rows*</u> | Traditional Import | Casting Data Type | Casting Data Type | Selected Column | Selected Column | Selected Column |
|-----------------------|--------------------|-------------------|-------------------|-----------------|-----------------|-----------------|
| **String Type**      | Object             | Object            | Category          | Object          | Object          | Category        |
| **Interger Type**           | Int 64             | Int 8             | Int 8             | Int 64          | Int 8           | Int 8           |
| **Execution time (secs)** | 4.65               | 3.82              | 8.58              | 2.58            | 2.12            | 2.45            |
| **Memory usage**          | 183.1+ MB          | 89.6+ MB          | 71.0 MB           | 114.4+ MB       | 34.3+ MB        | 15.3 MB         |
| **Total columns**         | 24                 | 24                | 24                | 15              | 15              | 15              |

| <u>*Import 1,000,000 rows*</u> | Traditional Import | Casting Data Type | Casting Data Type | Selected Column | Selected Column | Selected Column | Excel Opening | Number Opening |
|-----------------------|--------------------|-------------------|-------------------|-----------------|-----------------|-----------------|---------------|----------------|
| **String Type**           | Object             | Object            | Category          | Object          | Object          | Category        | -             | -              |
| **Interger Type**         | Int 64             | Int 8             | Int 8             | Int 64          | Int 8           | Int 8           | -             | -              |
| **Execution time (secs)** | 4.65               | 3.82              | 8.58              | 2.58            | 2.12            | 2.45            | 55.27         | 156.17         |
| **Memory usage**          | 183.1+ MB          | 89.6+ MB          | 71.0 MB           | 114.4+ MB       | 34.3+ MB        | 15.3 MB         | 157.1 MB      | 157.1 MB       |
| **Total columns**         | 24                 | 24                | 24                | 15              | 15              | 15              | 24            | 24             |

***

In [21]:
df.max()

id                  1.84467e+19
click                         1
hour                   14103023
C1                         1012
banner_pos                    7
site_id                fffe8e1c
site_domain            fff602a2
site_category          f66779e6
app_id                 ffdb187c
app_domain             fe369646
app_category           fc6fa53d
device_id              ffffa2c2
device_ip              fffffaa3
device_model           ffeafe15
device_type                   5
device_conn_type              5
C14                       24046
C15                        1024
C16                        1024
C17                        2757
C18                           3
C19                        1839
C20                      100248
C21                         255
dtype: object

In [24]:
df_sam_6.max()

click                      1
C1                       -12
banner_pos                 7
site_category       f66779e6
app_domain          fe369646
app_category        fc6fa53d
device_type                5
device_conn_type           5
C15                      120
C16                       90
C17                      127
C18                        3
C19                       47
C20                      125
C21                      126
dtype: object

In [29]:
print(np.int8(129))
print(np.int8(1024))

-127
0


In [31]:
dict_all

{'click': numpy.int8,
 'hour': numpy.int8,
 'C1': numpy.int8,
 'banner_pos': numpy.int8,
 'device_type': numpy.int8,
 'device_conn_type': numpy.int8,
 'C14': numpy.int8,
 'C15': numpy.int8,
 'C16': numpy.int8,
 'C17': numpy.int8,
 'C18': numpy.int8,
 'C19': numpy.int8,
 'C20': numpy.int8,
 'C21': numpy.int8,
 'site_id': 'category',
 'site_domain': 'category',
 'site_category': 'category',
 'app_id': 'category',
 'app_domain': 'category',
 'app_category': 'category',
 'device_id': 'category',
 'device_ip': 'category',
 'device_model': 'category'}

In [40]:
dict_all_16 = {i:np.int16 for i in Integer_cols} 
dict_cat = {i:'category' for i in Object_cols}
dict_all_16.update(dict_cat)
dict_all_16

{'click': numpy.int16,
 'hour': numpy.int16,
 'C1': numpy.int16,
 'banner_pos': numpy.int16,
 'device_type': numpy.int16,
 'device_conn_type': numpy.int16,
 'C14': numpy.int16,
 'C15': numpy.int16,
 'C16': numpy.int16,
 'C17': numpy.int16,
 'C18': numpy.int16,
 'C19': numpy.int16,
 'C20': numpy.int16,
 'C21': numpy.int16,
 'site_id': 'category',
 'site_domain': 'category',
 'site_category': 'category',
 'app_id': 'category',
 'app_domain': 'category',
 'app_category': 'category',
 'device_id': 'category',
 'device_ip': 'category',
 'device_model': 'category'}

# Selected Column with Casting int64 > int16

In [30]:
[2**i for i in [8,16,32,64]]

[256, 65536, 4294967296, 18446744073709551616]

In [36]:
dict_int_3 = {i:np.int16 for i in Integer_cols} 
# del dict_int_2['hour']
''' 
Deleting an entry from dictionary using del 
'''
# If key exist in dictionary then delete it using del.
for key in dict_int.keys():
    if key not in read_col:
        del dict_int_3[key]

In [42]:
start = time()
df_sam_6 = pd.read_csv('./data/avazu-ctr-prediction/sample1m.csv',
                       usecols = read_col,
                       dtype = dict_int_3,
#                        parse_dates=['hour'], date_parser = mydateparser,
                      )
t = time() - start
print('Execution time = ', t, ' seconds')
df_sam_6.info()

Execution time =  2.151221990585327  seconds
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 15 columns):
click               1000000 non-null int16
C1                  1000000 non-null int16
banner_pos          1000000 non-null int16
site_category       1000000 non-null object
app_domain          1000000 non-null object
app_category        1000000 non-null object
device_type         1000000 non-null int16
device_conn_type    1000000 non-null int16
C15                 1000000 non-null int16
C16                 1000000 non-null int16
C17                 1000000 non-null int16
C18                 1000000 non-null int16
C19                 1000000 non-null int16
C20                 1000000 non-null int16
C21                 1000000 non-null int16
dtypes: int16(12), object(3)
memory usage: 45.8+ MB


# Selected Column with Casting int64 > int16, object > category

In [41]:
start = time()
df_sam_6 = pd.read_csv('./data/avazu-ctr-prediction/sample1m.csv',
                       usecols = read_col,
                       dtype = dict_all_16,
#                        parse_dates=['hour'], date_parser = mydateparser,
                      )
t = time() - start
print('Execution time = ', t, ' seconds')
df_sam_6.info()

Execution time =  2.2928900718688965  seconds
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 15 columns):
click               1000000 non-null int16
C1                  1000000 non-null int16
banner_pos          1000000 non-null int16
site_category       1000000 non-null category
app_domain          1000000 non-null category
app_category        1000000 non-null category
device_type         1000000 non-null int16
device_conn_type    1000000 non-null int16
C15                 1000000 non-null int16
C16                 1000000 non-null int16
C17                 1000000 non-null int16
C18                 1000000 non-null int16
C19                 1000000 non-null int16
C20                 1000000 non-null int16
C21                 1000000 non-null int16
dtypes: category(3), int16(12)
memory usage: 26.7 MB


|<u>*Import 1,000,000 rows*</u>| Traditional Import | Casting Data Type | Casting Data Type | Selected Column | Selected Column | Selected Column | Selected Column | Selected Column |
|-----------------------|--------------------|-------------------|-------------------|-----------------|-----------------|-----------------|-----------------|-----------------|
| **String Type**           | Object             | Object            | Category          | Object          | Object          | Category        | Object          | Category        |
| **Interger Type**         | Int 64             | Int 8             | Int 8             | Int 64          | Int 8           | Int 8           | int16           | int16           |
| **Execution time (secs)** | 4.65               | 3.82              | 8.58              | 2.58            | 2.12            | 2.45            | 2.15            | 2.43            |
| **Memory usage**          | 183.1+ MB          | 89.6+ MB          | 71.0 MB           | 114.4+ MB       | 34.3+ MB        | 15.3 MB         | 45.8+ MB        | 26.7 MB         |
| **Total columns**         | 24                 | 24                | 24                | 15              | 15              | 15              | 15              | 15              |